In [1]:
from llama_stack_client import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
import sys
sys.path.append('..') 
from src.client_tools import get_location
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# pretty print of the results returned from the model/agent
import sys
sys.path.append('..')  
from src.utils import step_printer
from termcolor import cprint

In [2]:
base_url = os.getenv("REMOTE_BASE_URL")

client = LlamaStackClient(
    base_url=base_url
)

# model_id will later be used to pass the name of the desired inference model to Llama Stack Agents/Inference APIs
model_id = "granite-3.2-2b-instruct" # "Llama-3.2-1B-Instruct-Q8_0.gguf" # "/mnt/models/DeepSeek-R1-0528-Qwen3-8B-Q4_K_M.gguf"  # "granite-3.2-2b-instruct" # 

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Inference Parameters:
	Model: granite-3.2-2b-instruct
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 15000}
	stream: True


In [3]:
# Optional: Enter your MCP server URL here
ocp_mcp_url = os.getenv("REMOTE_OCP_MCP_URL") # Optional: enter your MCP server url here

# Get list of registered tools and extract their toolgroup IDs
registered_tools = client.tools.list()
registered_toolgroups = [tool.toolgroup_id for tool in registered_tools]

if "mcp::openshift" not in registered_toolgroups: # required
    client.toolgroups.register(
        toolgroup_id="mcp::openshift",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":ocp_mcp_url},
    )

In [32]:
agent = ReActAgent(
            client=client,
            model=model_id,
            tools=["mcp::openshift"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            sampling_params=sampling_params,
        )

user_prompts = ["""Create namespace=foo, namespace=bar the cluster.
                """
                ]

session_id = agent.create_session("web-session")
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Create namespace=foo, namespace=bar the cluster.
                
inference> {
    "thought": "I need to create two namespaces, 'foo' and 'bar', in the current Kubernetes cluster. I will use the 'namespace_create' tool for this task.",
    "action": {
        "tool_name": "namespace_create",
        "tool_params": [
            {"name": "namespace", "value": "foo"},
            {"name": "namespace", "value": "bar"}
        ]
    },
    "answer": null
}
tool_execution> Tool:namespace_create Response:{"type":"text","text":"bar","annotations":null}
inference> {
    "thought": "The 'namespace_create' tool has successfully created the namespace 'bar'. Now, I need to create the namespace 'foo'.",
    "action": {
        "tool_name": "namespace_create",
        "tool_params": [
            {"name": "namespace", "value": "foo"}
        ]
    },
    "answer": null
}
tool_execution> Tool:namespace_create Response:{"type":"text","text":"foo","annotations":null}
inference> 